## Overview Analysis
![img.png](img/SystemStartUp.png)

In [17]:
package 'Power Roll Up'{
    import ScalarValues::Real;
    import ScalarFunctions::sum;
    
    import 'Roll-Up Pattern'::*;
    import Behavior::*;
    import Structure::*;
    import Analysis_package::*;
    
    value type Power specializes Real;
    
    
    package 'Roll-Up Pattern' {
        block PowerConstraint {
            
            exhibit behavior : 'PRBehavior' as behavior; // What is the 'as' keyword?
            
            part subPower : PowerConstraint [*];
            value power : Power = 0.0;
            value operatingPower : Power = 0.0;
            value initPower : Power = 0.0;
            value totalPower : Power = sum(subPower::totalPower) + power;
        }
        
        block SystemWithCBE specializes System, PowerConstraint {
            value redefines operatingPower = 0;
            value redefines initPower = 0;
            block Motor1 specializes PowerConstraint;
            block Motor2 specializes PowerConstraint;
            block Motor3 specializes PowerConstraint;
            
            // Should the values be redefined at the block level?
            // Is there even a need for 3 different blocks?
            part redefines motor1 : Motor1 subsets subPower{ 
                value redefines operatingPower = 30;
                value redefines initPower = 50;
            }
            part redefines motor2 : Motor2 subsets subPower{
                value redefines operatingPower = 20;
                value redefines initPower = 15;
            }
            part redefines motor3 : Motor3 subsets subPower{
                value redefines operatingPower = 5;
                value redefines initPower = 10;
            }
        }
    } // END package 'Roll-Up Pattern'
    
    package Analysis_package {
        block Analysis {
            constraint def Comparison(p1: Real, p2: Real){
                p1 <= p2
            }
            block Driver;

            part system : SystemWithCBE;
            part driver : Driver;
            value max : Power = 100.0;
            constraint comp : Comparison(p1=system::totalPower, p2=max);
        } // END block Analysis
    } // END package Analysis
    
    
    package Behavior {
        value type Done;
        value type Init;
        value type TurnOff;
        value type TurnOn;
        
        state def 'PRBehavior' {    
            entry; then Off;

            state Off {
                entry action{
                    power = 0;
                }
            }
            transition off_to_initializing
                first Off
                accept Init
                then initializing;
            transition off_to_on
                first Off
                accept TurnOn
                then On;

            state initializing {
                entry action{
                    power = PowerConstraint::initPower;
                }
            }
            transition initializing_to_off
                first initializing
                accept Done
                then Off;

            state On {
                entry action{
                    power = PowerConstraint::operatingPower;
                }
            }
            transition on_to_initializing
                first On
                accept Init
                then initializing;
            transition on_to_off
                first On
                accept TurnOff
                then Off;
        } // END state def 'PRBehavior'
        
        activity AnalysisResult {
            first start;
            then fork parallel; 
            // This needs strict concurrent execution (e.g. s1.startTime==s2.startTime)
            succession parallel then s1;
            succession parallel then s2;

            action s1 : SendInit;
            then join_node;

            action s2 : SendTurnOn;
            then action s3 : SendTurnOff;
            then join_node;

            join join_node;
            then done;
        }

        activity AnalysisInput specializes TimedActivity{
            // Constraint s1.startTime==s2.startTime;
            action s1 : SendInit;

            action s2 : SendTurnOn;
            then action s3 : SendTurnOff;
        }

        activity SendInit specializes TimedActivity{
            first start;
            then send SendInit
                of Init()
                to Analysis::system::motor1; // Not 100% where to send the signal. Is there a way to send to "public"?
            then done;
        }
        activity SendTurnOff specializes TimedActivity{
            first start;
            then send SendTurnOff
                of TurnOff()
                to Analysis::system::motor2;
            then done;
        }
        activity SendTurnOn specializes TimedActivity{
            first start;
            then send SendTurnOn
                of TurnOn()
                to Analysis::system::motor3;
            then done;
        }
        
        activity TimedActivity {
            value startTime : Real;
            value endTime : Real;
            value duration : Real;
            ref subActivity : TimedActivity; //not sure if that should be a ref?
        }
    }
    
    package Structure {
        block System {
            block Motor1;
            block Motor2;
            block Motor3;
            part motor3 : Motor3;
            part motor2 : Motor2;
            part motor1 : Motor1;
        }
    }
}

Package Power Roll Up (664fb138-afe9-4a7e-8b4b-b10fee416a35)
